# GuardRails Validation Traces Demo

This notebook demonstrates **GuardRails** - declarative validators with rich trace generation for agent transparency, inspired by Guardrails AI.

## What You'll Learn
1. Creating declarative validation constraints
2. Validating agent outputs with guardrails
3. Using built-in validators (PII, length, regex, etc.)
4. Generating validation traces for debugging


In [1]:
# Setup imports
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent))

from backend.explainability.guardrails import (
    GuardRail,
    GuardRailValidator,
    BuiltInValidators,
    FailAction,
)

# Create a guardrail
guardrail = GuardRail(
    name="invoice_validator",
    description="Validates invoice extraction outputs",
    constraints=[
        BuiltInValidators.no_pii(),
        BuiltInValidators.required_fields(["vendor", "amount"]),
    ],
)

# Validate data
validator = GuardRailValidator()
result = validator.validate(
    {"vendor": "Acme Corp", "amount": 100, "output": "Valid data"},
    guardrail
)

print(f"Validation result: {'VALID' if result.is_valid else 'INVALID'}")
print(f"Errors: {result.total_errors}, Warnings: {result.total_warnings}")


Validation result: VALID
Errors: 0, Warnings: 0


## 2. Creating Declarative Constraints

Constraints are the building blocks of guardrails. Each constraint defines:
- **name**: Unique identifier
- **description**: Human-readable explanation
- **check_fn**: The validation function to execute
- **severity**: ERROR (blocking), WARNING (non-blocking), or INFO (informational)
- **on_fail**: Action to take - REJECT, FIX, ESCALATE, LOG, or RETRY


In [2]:
from backend.explainability.guardrails import Constraint, Severity

# Create individual constraints with different severity levels

# ERROR severity - validation fails if this doesn't pass
ssn_check = Constraint(
    name="no_ssn",
    description="Output must not contain Social Security Numbers",
    check_fn="pii",  # Maps to BuiltInValidators.check_pii
    params={},
    severity=Severity.ERROR,
    on_fail=FailAction.REJECT,
)

# WARNING severity - logs warning but doesn't fail validation
length_warning = Constraint(
    name="reasonable_length",
    description="Output should be between 10 and 500 characters",
    check_fn="length",
    params={"min_len": 10, "max_len": 500},
    severity=Severity.WARNING,
    on_fail=FailAction.LOG,
)

# INFO severity - purely informational
format_info = Constraint(
    name="json_format_check",
    description="Check if output is valid JSON (informational)",
    check_fn="json",
    params={},
    severity=Severity.INFO,
    on_fail=FailAction.LOG,
)

print("Created constraints:")
print(f"  - {ssn_check.name}: severity={ssn_check.severity.value}, on_fail={ssn_check.on_fail.value}")
print(f"  - {length_warning.name}: severity={length_warning.severity.value}, on_fail={length_warning.on_fail.value}")
print(f"  - {format_info.name}: severity={format_info.severity.value}, on_fail={format_info.on_fail.value}")


Created constraints:
  - no_ssn: severity=error, on_fail=reject
  - reasonable_length: severity=warning, on_fail=log
  - json_format_check: severity=info, on_fail=log


## 3. PII Detection with Real Data

Let's use the `pii_examples_50.json` dataset to test PII detection. This dataset contains examples with SSN, credit cards, emails, phone numbers, and more.


In [3]:
import json

# Load PII examples dataset
data_path = Path.cwd().parent / "data" / "pii_examples_50.json"
with open(data_path) as f:
    pii_examples = json.load(f)

print(f"Loaded {len(pii_examples)} PII examples")
print(f"PII types in dataset: {set(t for ex in pii_examples for t in ex['pii_types'])}")

# Create a PII detection guardrail
pii_guardrail = GuardRail(
    name="pii_detector",
    description="Detects and rejects outputs containing PII",
    constraints=[BuiltInValidators.no_pii()],
    on_fail_default=FailAction.REJECT,
)

# Test with samples containing different PII types
test_samples = [
    pii_examples[9],   # SSN example
    pii_examples[1],   # Credit card example  
    pii_examples[4],   # Email + phone example
]

print("\n--- PII Detection Results ---")
for sample in test_samples:
    result = validator.validate({"output": sample["text"]}, pii_guardrail)
    status = "BLOCKED" if not result.is_valid else "PASSED"
    print(f"\n[{status}] PII types: {sample['pii_types']}")
    print(f"  Text: {sample['text'][:60]}...")
    if not result.is_valid:
        print(f"  Reason: {result.entries[0].message}")


Loaded 50 PII examples
PII types in dataset: {'name', 'passport', 'phone', 'email', 'drivers_license', 'credit_card', 'medical_record_number', 'ssn'}

--- PII Detection Results ---

[BLOCKED] PII types: ['name', 'ssn']
  Text: Loan application: John Gonzalez, SSN 490-86-8668. Annual inc...
  Reason: Potential SSN detected

[BLOCKED] PII types: ['name', 'credit_card', 'passport']
  Text: Hotel reservation: Mary Davis, Passport #P80823176. Card: 24...
  Reason: Potential credit card number detected

[BLOCKED] PII types: ['phone', 'name', 'email']
  Text: User Matthew Wilson requests password reset. Email: mattheww...
  Reason: Email address detected


## 4. Built-in Validators Demo

The `BuiltInValidators` class provides factory methods for common validation patterns:
- `length_check(min, max)` - Validate string length
- `regex_match(pattern)` - Pattern matching
- `required_fields([fields])` - Check mandatory fields
- `confidence_range(min, max)` - Validate numeric ranges
- `json_parseable()` - Ensure valid JSON
- `value_in_list(values)` - Enum-style validation


In [4]:
# Demo each built-in validator

# 1. Length Check
length_guardrail = GuardRail(
    name="length_validator",
    description="Validates output length",
    constraints=[BuiltInValidators.length_check(min_len=5, max_len=100)],
)

# Test cases
test_cases = [
    {"output": "Hi"},           # Too short
    {"output": "Valid output"}, # Just right
    {"output": "x" * 150},      # Too long
]

print("--- Length Check ---")
for tc in test_cases:
    result = validator.validate(tc, length_guardrail)
    print(f"  '{tc['output'][:20]}...' (len={len(tc['output'])}): {result.entries[0].message}")

# 2. Regex Match (invoice number pattern)
regex_guardrail = GuardRail(
    name="invoice_format",
    description="Validates invoice number format",
    constraints=[BuiltInValidators.regex_match(r"INV-\d{4}-\d{4}")],
)

print("\n--- Regex Match (Invoice Format INV-XXXX-XXXX) ---")
for inv in ["INV-2024-0001", "INVOICE-123", "inv-2024-0001"]:
    result = validator.validate({"output": inv}, regex_guardrail)
    status = "MATCH" if result.is_valid else "NO MATCH"
    print(f"  '{inv}': {status}")

# 3. Confidence Range
confidence_guardrail = GuardRail(
    name="confidence_check",
    description="Ensures confidence scores are valid",
    constraints=[BuiltInValidators.confidence_range(min_conf=0.7, max_conf=1.0)],
)

print("\n--- Confidence Range (0.7 - 1.0) ---")
for conf in [0.95, 0.65, 1.2]:
    result = validator.validate({"confidence": conf}, confidence_guardrail)
    status = "VALID" if result.is_valid else "INVALID"
    print(f"  confidence={conf}: {status} - {result.entries[0].message}")

# 4. Value in List
category_guardrail = GuardRail(
    name="category_validator",
    description="Validates document category",
    constraints=[BuiltInValidators.value_in_list(
        allowed_values=["invoice", "receipt", "contract", "statement"],
        field="category"
    )],
)

print("\n--- Value in List (Document Categories) ---")
for cat in ["invoice", "receipt", "memo"]:
    result = validator.validate({"category": cat}, category_guardrail)
    status = "VALID" if result.is_valid else "INVALID"
    print(f"  category='{cat}': {status}")


--- Length Check ---
  'Hi...' (len=2): Length 2 is below minimum 5
  'Valid output...' (len=12): Length 12 is within bounds [5, 100]
  'xxxxxxxxxxxxxxxxxxxx...' (len=150): Length 150 exceeds maximum 100

--- Regex Match (Invoice Format INV-XXXX-XXXX) ---
  'INV-2024-0001': MATCH
  'INVOICE-123': NO MATCH
  'inv-2024-0001': NO MATCH

--- Confidence Range (0.7 - 1.0) ---
  confidence=0.95: VALID - Confidence 0.95 is within range [0.7, 1.0]
  confidence=0.65: INVALID - Confidence 0.65 is below minimum 0.7
  confidence=1.2: INVALID - Confidence 1.2 exceeds maximum 1.0

--- Value in List (Document Categories) ---
  category='invoice': VALID
  category='receipt': VALID
  category='memo': INVALID


## 5. Combining Multiple Validators

Real-world guardrails combine multiple constraints. Here we create a comprehensive invoice extraction validator.


In [5]:
# Create a comprehensive invoice extraction guardrail
invoice_guardrail = GuardRail(
    name="invoice_extraction_v2",
    description="Validates invoice extraction agent outputs",
    version="2.0.0",
    constraints=[
        BuiltInValidators.no_pii(),
        BuiltInValidators.required_fields(["vendor_name", "invoice_number", "total_amount"]),
        BuiltInValidators.confidence_range(min_conf=0.8, max_conf=1.0),
        BuiltInValidators.length_check(min_len=10, max_len=5000),
    ],
    on_fail_default=FailAction.REJECT,
)

# Test with different scenarios
test_scenarios = [
    {
        "name": "Valid extraction",
        "data": {
            "vendor_name": "Acme Corporation",
            "invoice_number": "INV-2024-0042",
            "total_amount": 1250.00,
            "confidence": 0.95,
            "output": "Invoice from Acme Corporation, total $1,250.00",
        },
    },
    {
        "name": "Missing required field",
        "data": {
            "vendor_name": "Tech Solutions",
            "total_amount": 500.00,
            "confidence": 0.88,
            "output": "Invoice from Tech Solutions",
        },
    },
    {
        "name": "Low confidence",
        "data": {
            "vendor_name": "Global Services",
            "invoice_number": "INV-2024-0099",
            "total_amount": 750.00,
            "confidence": 0.65,
            "output": "Invoice extraction uncertain",
        },
    },
    {
        "name": "Contains PII (SSN)",
        "data": {
            "vendor_name": "Payroll Inc",
            "invoice_number": "INV-2024-0100",
            "total_amount": 2000.00,
            "confidence": 0.92,
            "output": "Employee SSN: 123-45-6789",
        },
    },
]

print("--- Combined Validator Results ---\n")
for scenario in test_scenarios:
    result = validator.validate(scenario["data"], invoice_guardrail)
    status = "PASS" if result.is_valid else "FAIL"
    print(f"[{status}] {scenario['name']}")
    print(f"   Errors: {result.total_errors}, Warnings: {result.total_warnings}")
    
    # Show failed constraints
    for entry in result.entries:
        if not entry.passed:
            print(f"   - {entry.constraint_name}: {entry.message}")
    print()


--- Combined Validator Results ---

[PASS] Valid extraction
   Errors: 0, Warnings: 0

[FAIL] Missing required field
   Errors: 1, Warnings: 0
   - required_fields: Missing required fields: invoice_number

[FAIL] Low confidence
   Errors: 1, Warnings: 0
   - confidence_range: Confidence 0.65 is below minimum 0.8

[FAIL] Contains PII (SSN)
   Errors: 1, Warnings: 0
   - no_pii: Potential SSN detected



## 6. PromptGuardRail for LLM Outputs

`PromptGuardRail` extends the base guardrail with prompt-specific fields for documenting LLM interactions. It includes prompt templates, required/optional fields, and example outputs.


In [6]:
from backend.explainability.guardrails import PromptGuardRail

# Create a prompt guardrail for invoice extraction LLM
prompt_guardrail = PromptGuardRail(
    name="invoice_extraction_prompt",
    description="Guardrail for invoice extraction LLM prompt",
    version="1.0.0",
    prompt_template="""Extract invoice data from the following text.
Return a JSON object with these fields:
- vendor_name: The name of the vendor
- invoice_number: The invoice identifier
- total_amount: The total amount as a number
- line_items: Array of items (optional)

Invoice text:
{invoice_text}""",
    required_fields=["vendor_name", "invoice_number", "total_amount"],
    optional_fields=["line_items", "due_date", "tax_amount"],
    example_valid_output='{"vendor_name": "Acme Corp", "invoice_number": "INV-001", "total_amount": 1500.00}',
    example_invalid_output='{"vendor": "Acme"}',  # Missing required fields
    constraints=[
        BuiltInValidators.json_parseable(),
        BuiltInValidators.no_pii(),
    ],
)

# Simulate LLM outputs
llm_outputs = [
    '{"vendor_name": "Tech Solutions", "invoice_number": "INV-2024-001", "total_amount": 2500.00}',
    '{"vendor_name": "Bad Corp"}',  # Missing fields
    'This is not JSON at all',      # Invalid JSON
    '{"vendor_name": "Payroll", "invoice_number": "INV-002", "total_amount": 1000, "notes": "SSN: 123-45-6789"}',
]

print("--- PromptGuardRail Validation ---\n")
for i, output in enumerate(llm_outputs, 1):
    result = validator.validate_prompt_output(output, prompt_guardrail)
    status = "VALID" if result.is_valid else "INVALID"
    print(f"Output {i}: [{status}]")
    print(f"  Preview: {output[:50]}...")
    print(f"  Errors: {result.total_errors}, Action: {result.action_taken}")
    
    for entry in result.entries:
        if not entry.passed:
            print(f"    - {entry.constraint_name}: {entry.message}")
    print()


--- PromptGuardRail Validation ---

Output 1: [VALID]
  Preview: {"vendor_name": "Tech Solutions", "invoice_number"...
  Errors: 0, Action: None

Output 2: [INVALID]
  Preview: {"vendor_name": "Bad Corp"}...
  Errors: 2, Action: FailAction.REJECT
    - required_field_invoice_number: Required field 'invoice_number' is missing
    - required_field_total_amount: Required field 'total_amount' is missing

Output 3: [INVALID]
  Preview: This is not JSON at all...
  Errors: 1, Action: FailAction.REJECT
    - json_parseable: Invalid JSON: Expecting value: line 1 column 1 (char 0)

Output 4: [INVALID]
  Preview: {"vendor_name": "Payroll", "invoice_number": "INV-...
  Errors: 1, Action: FailAction.REJECT
    - no_pii: Potential SSN detected



## 7. Validation Traces

The validator maintains a trace of all validation entries. This is invaluable for debugging, auditing, and understanding why validations passed or failed.


In [7]:
# Get all validation entries from this session
trace = validator.get_validation_trace()

print(f"Total validation entries in trace: {len(trace)}\n")

# Analyze trace statistics
passed = sum(1 for e in trace if e.passed)
failed = len(trace) - passed

print("--- Trace Summary ---")
print(f"  Passed: {passed}")
print(f"  Failed: {failed}")
print(f"  Pass rate: {passed/len(trace)*100:.1f}%\n")

# Group by constraint name
from collections import Counter
constraint_counts = Counter(e.constraint_name for e in trace)
print("--- Constraints Checked ---")
for name, count in constraint_counts.most_common(5):
    print(f"  {name}: {count} times")

# Show recent failed validations
print("\n--- Recent Failed Validations ---")
failed_entries = [e for e in trace if not e.passed][-5:]
for entry in failed_entries:
    print(f"  [{entry.severity.value.upper()}] {entry.constraint_name}")
    print(f"    {entry.message}")
    if entry.input_excerpt:
        print(f"    Input: {entry.input_excerpt[:60]}...")


Total validation entries in trace: 50

--- Trace Summary ---
  Passed: 33
  Failed: 17
  Pass rate: 66.0%

--- Constraints Checked ---
  no_pii: 12 times
  length_check: 7 times
  confidence_range: 7 times
  required_fields: 5 times
  json_parseable: 4 times

--- Recent Failed Validations ---
  [ERROR] no_pii
    Potential SSN detected
    Input: {'vendor_name': 'Payroll Inc', 'invoice_number': 'INV-2024-0...
  [ERROR] required_field_invoice_number
    Required field 'invoice_number' is missing
    Input: {"vendor_name": "Bad Corp"}...
  [ERROR] required_field_total_amount
    Required field 'total_amount' is missing
    Input: {"vendor_name": "Bad Corp"}...
  [ERROR] json_parseable
    Invalid JSON: Expecting value: line 1 column 1 (char 0)
    Input: {'output': 'This is not JSON at all'}...
  [ERROR] no_pii
    Potential SSN detected
    Input: {'output': '{"vendor_name": "Payroll", "invoice_number": "IN...


In [8]:
# Export trace to JSON for auditing
cache_dir = Path.cwd().parent / "cache" / "guardrails_demo"
cache_dir.mkdir(parents=True, exist_ok=True)

trace_file = cache_dir / "validation_trace.json"
validator.export_trace(trace_file)

print(f"Trace exported to: {trace_file}")

# Preview the exported file
with open(trace_file) as f:
    exported = json.load(f)
    
print(f"\nExported trace contains {exported['entry_count']} entries")
print(f"Exported at: {exported['exported_at']}")
print("\nSample entry:")
print(json.dumps(exported['entries'][0], indent=2, default=str))


Trace exported to: /Users/rajnishkhatri/Documents/recipe-chatbot/lesson-17/cache/guardrails_demo/validation_trace.json

Exported trace contains 50 entries
Exported at: 2025-11-27T18:23:56.958174+00:00

Sample entry:
{
  "constraint_name": "no_pii",
  "passed": true,
  "message": "No PII patterns detected",
  "severity": "error",
  "timestamp": "2025-11-27T18:23:56.906696Z",
  "input_excerpt": null,
  "fix_applied": null
}


## 8. Documentation Generation

Guardrails can self-document their constraints. This is useful for generating API documentation, compliance reports, and developer guides.


In [9]:
# Generate documentation for the invoice guardrail
print("=== Invoice Extraction Guardrail Documentation ===\n")
print(invoice_guardrail.document())


=== Invoice Extraction Guardrail Documentation ===

# invoice_extraction_v2

**Version:** 2.0.0

**Description:** Validates invoice extraction agent outputs

## Constraints

### no_pii
- **Description:** Output must not contain PII (SSN, credit card, etc.)
- **Severity:** error
- **On Fail:** reject

### required_fields
- **Description:** Required fields: vendor_name, invoice_number, total_amount
- **Severity:** error
- **On Fail:** reject
- **Params:** `{"fields": ["vendor_name", "invoice_number", "total_amount"]}`

### confidence_range
- **Description:** Confidence must be between 0.8 and 1.0
- **Severity:** error
- **On Fail:** reject
- **Params:** `{"min_conf": 0.8, "max_conf": 1.0}`

### length_check
- **Description:** String length must be between 10 and 5000
- **Severity:** error
- **On Fail:** reject
- **Params:** `{"min_len": 10, "max_len": 5000}`




In [10]:
# PromptGuardRail documentation includes prompt template and examples
print("=== Prompt GuardRail Documentation ===\n")
print(prompt_guardrail.document())


=== Prompt GuardRail Documentation ===

# invoice_extraction_prompt

**Version:** 1.0.0

**Description:** Guardrail for invoice extraction LLM prompt

## Constraints

### json_parseable
- **Description:** Output must be valid JSON
- **Severity:** error
- **On Fail:** reject

### no_pii
- **Description:** Output must not contain PII (SSN, credit card, etc.)
- **Severity:** error
- **On Fail:** reject

## Prompt Template

```
Extract invoice data from the following text.
Return a JSON object with these fields:
- vendor_name: The name of the vendor
- invoice_number: The invoice identifier
- total_amount: The total amount as a number
- line_items: Array of items (optional)

Invoice text:
{invoice_text}
```

## Required Fields

- `vendor_name`
- `invoice_number`
- `total_amount`

## Optional Fields

- `line_items`
- `due_date`
- `tax_amount`

## Example Valid Output

```json
{"vendor_name": "Acme Corp", "invoice_number": "INV-001", "total_amount": 1500.00}
```

## Example Invalid Output

``

## Summary

In this notebook, we demonstrated:

1. **Declarative Constraints** - Creating validation rules with different severity levels and failure actions
2. **PII Detection** - Using real data from `pii_examples_50.json` to validate PII detection
3. **Built-in Validators** - Length checks, regex matching, required fields, confidence ranges, and value lists
4. **Combined Validators** - Chaining multiple constraints for comprehensive validation
5. **PromptGuardRail** - Extended guardrails for LLM prompt/output validation with templates and examples
6. **Validation Traces** - Inspecting and exporting validation history for debugging and auditing
7. **Self-Documentation** - Generating markdown documentation from guardrail definitions

These patterns enable transparent, auditable AI agent validation with rich debugging capabilities.
